In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
import holidays
import datetime as dt
os.getcwd()

'/Users/miek/Documents/DS4B/DS4BThesisSNCB'

In [6]:
# Read tjj
stops = pd.read_parquet("~/Downloads/tjj_allptcars")
stops = stops[stops["REF_REPERAGE_NO"] > 0]
stops['TRAIN_DEPARTURE_DATE'] = pd.to_datetime(stops['TRAIN_DEPARTURE_DATE'])
stops

,REF_TOD_TRAIN_ID,TRAIN_PREFIX,TRAIN_NUMBER,TRAIN_DEPARTURE_DATE,REF_PTCAR_NO,PTCAR_ORD_NO,THR_ARR,THR_DEP,HR_ARR,HR_DEP,EH_ARR,EH_DEP,REF_REPERAGE_NO,DEPARTURE_AT_DT_ORIGIN,ARRIVAL_AT_DT_ORIGIN,COMMERCIAL_STOP_NATURE,DEPARTURE_REF_LINE_SECTION_NO,LINE,IDENTIFICATOR
0,31520427,None,2067,2022-11-23,6,124,2022-11-23 18:54:00,1900-01-01 00:00:00,2022-11-23 18:53:24,1900-01-01 00:00:00,-35,0,1,1900-01-01 00:00:00,2022-11-23 18:54:00,3,0,NaN,None
2,31646284,None,2673,2022-12-12,1840,102,2022-12-12 23:21:00,2022-12-12 23:21:00,2022-12-12 23:21:51,2022-12-12 23:21:51,51,51,1,2022-12-12 23:21:00,2022-12-12 23:21:00,0,1410,59.0,59/1
3,30430608,None,2835,2022-06-05,518,133,2022-06-05 15:09:00,2022-06-05 15:09:00,2022-06-05 15:09:01,2022-06-05 15:09:01,1,1,1,2022-06-05 15:09:00,2022-06-05 15:09:00,0,1948,50.0,50A
4,31239066,None,2169,2022-10-17,883,104,2022-10-17 19:45:00,2022-10-17 19:45:00,2022-10-17 19:44:59,2022-10-17 19:44:59,0,0,1,2022-10-17 19:45:00,2022-10-17 19:45:00,0,1439,27.0,27B
5,30285266,None,1827,2022-05-12,37,101,1900-01-01 00:00:00,2022-05-12 06:06:00,1900-01-01 00:00:00,2022-05-12 06:13:37,0,457,1,2022-05-12 06:06:00,1900-01-01 00:00:00,1,1410,59.0,59/1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36044908,32467093,None,4692,2023-05-03,848,114,2023-05-03 22:08:00,1900-01-01 00:00:00,2023-05-03 22:08:57,1900-01-01 00:00:00,57,0,1,1900-01-01 00:00:00,2023-05-03 22:08:00,3,0,NaN,None
36044909,32466051,None,2409,2023-05-03,734,172,2023-05-03 11:41:00,1900-01-01 00:00:00,2023-05-03 11:46:04,1900-01-01 00:00:00,304,0,1,1900-01-01 00:00:00,2023-05-03 11:41:00,0,0,NaN,None
36044910,32420846,None,7868,2023-04-25,848,114,2023-04-25 07:53:00,1900-01-01 00:00:00,2023-04-25 07:56:48,1900-01-01 00:00:00,228,0,1,1900-01-01 00:00:00,2023-04-25 07:53:00,3,0,NaN,None
36044911,32420498,None,6059,2023-04-25,723,121,2023-04-25 11:47:00,1900-01-01 00:00:00,2023-04-25 11:44:48,1900-01-01 00:00:00,-131,0,1,1900-01-01 00:00:00,2023-04-25 11:47:00,3,0,NaN,None


In [7]:
stops.dtypes

REF_TOD_TRAIN_ID                          int64
TRAIN_PREFIX                             object
TRAIN_NUMBER                              int64
TRAIN_DEPARTURE_DATE             datetime64[ns]
REF_PTCAR_NO                              int64
PTCAR_ORD_NO                              int32
THR_ARR                          datetime64[ns]
THR_DEP                          datetime64[ns]
HR_ARR                           datetime64[ns]
HR_DEP                           datetime64[ns]
EH_ARR                                    int64
EH_DEP                                    int64
REF_REPERAGE_NO                           int16
DEPARTURE_AT_DT_ORIGIN           datetime64[ns]
ARRIVAL_AT_DT_ORIGIN             datetime64[ns]
COMMERCIAL_STOP_NATURE                    int16
DEPARTURE_REF_LINE_SECTION_NO             int64
LINE                                    float64
IDENTIFICATOR                            object
dtype: object

In [8]:
# add year column
stops["YEAR_DEP"] = stops["TRAIN_DEPARTURE_DATE"].dt.year

# intermediate df no. 1: stops filtered only on the final station per train
ending_stop = (stops.sort_values('THR_ARR', ascending=False)
               .drop_duplicates('REF_TOD_TRAIN_ID')
               .reset_index(drop=True)
)

# add column for datetime of the end of each train
ending_stop['end_lifetime'] = ending_stop['THR_ARR']

# create a new column with only the hour component of theoretical departure time
ending_stop['arr_hour_terminus'] = ending_stop['THR_ARR'].dt.hour + ending_stop['THR_ARR'].dt.minute / 60


# intermediate df no. 2: stops filtered only on departure station per train
starting_stop = (stops[(stops["THR_DEP"].dt.year != 1900)]
                 .sort_values('THR_DEP', ascending=True)
                 .drop_duplicates('REF_TOD_TRAIN_ID')
                 .reset_index(drop=True)
)

# add column for datetime of the start of each train
starting_stop['start_lifetime'] = starting_stop['THR_DEP']

# create a new column with only the hour component of theoretical departure time
starting_stop['dep_hour_origin'] = starting_stop['THR_DEP'].dt.hour + starting_stop['THR_DEP'].dt.minute / 60

# Create date column
starting_stop['date_origin'] = starting_stop['THR_DEP'].dt.date
starting_stop['date_origin'] = pd.to_datetime(starting_stop['date_origin'])

# Create month column
starting_stop['month'] = starting_stop['THR_DEP'].dt.month

# Create day column
starting_stop['day_origin'] = starting_stop['THR_DEP'].dt.day

# Create season column
starting_stop['season'] = starting_stop['month'].apply(lambda x:
                                                       'Winter' if x <= 2 else 'Spring' if x <= 5 else 'Summer' if x <= 8 else 'Fall' if x <= 11 else 'Winter')

# Create weekday/weekend column
starting_stop['weekend'] = np.where((starting_stop['THR_DEP']).dt.weekday < 5, 0, 1)

# Create Belgian national holiday dummy column
be_holidays = holidays.BE()
starting_stop['holiday'] = starting_stop['THR_DEP'].apply(lambda x: 1 if x in be_holidays else 0)




# merge origin departure time to all stops a train will make
stops = stops.merge(starting_stop[['REF_TOD_TRAIN_ID', 'month', 'season',
                                   'weekend', 'holiday', 'dep_hour_origin',
                                   'day_origin', 'date_origin', 'start_lifetime']],
                    on='REF_TOD_TRAIN_ID',
                    how='left')\
             .merge(ending_stop[['REF_TOD_TRAIN_ID', 'end_lifetime', 'arr_hour_terminus']],
                    on='REF_TOD_TRAIN_ID',
                    how='left')

# reduce columns of stops to useful features
stops = stops.drop(columns=["TRAIN_PREFIX", "REF_REPERAGE_NO", "DEPARTURE_AT_DT_ORIGIN", "ARRIVAL_AT_DT_ORIGIN"])

# add column for P-train
stops["p_indicator"] = stops['TRAIN_NUMBER'].apply(lambda x: 1 if ((x >= 7000) & (x <= 8999)) else 0)


# check
stops

,REF_TOD_TRAIN_ID,TRAIN_NUMBER,TRAIN_DEPARTURE_DATE,REF_PTCAR_NO,PTCAR_ORD_NO,THR_ARR,THR_DEP,HR_ARR,HR_DEP,EH_ARR,...,season,weekend,holiday,dep_hour_origin,day_origin,date_origin,start_lifetime,end_lifetime,arr_hour_terminus,p_indicator
0,31520427,2067,2022-11-23,6,124,2022-11-23 18:54:00,1900-01-01 00:00:00,2022-11-23 18:53:24,1900-01-01 00:00:00,-35,...,Fall,0.0,0.0,17.916667,23.0,2022-11-23,2022-11-23 17:55:00,2022-11-23 18:54:00,18.900000,0
1,31646284,2673,2022-12-12,1840,102,2022-12-12 23:21:00,2022-12-12 23:21:00,2022-12-12 23:21:51,2022-12-12 23:21:51,51,...,Winter,0.0,0.0,23.316667,12.0,2022-12-12,2022-12-12 23:19:00,2022-12-12 23:50:00,23.833333,0
2,30430608,2835,2022-06-05,518,133,2022-06-05 15:09:00,2022-06-05 15:09:00,2022-06-05 15:09:01,2022-06-05 15:09:01,1,...,Summer,1.0,1.0,13.950000,5.0,2022-06-05,2022-06-05 13:57:00,2022-06-05 15:54:00,15.900000,0
3,31239066,2169,2022-10-17,883,104,2022-10-17 19:45:00,2022-10-17 19:45:00,2022-10-17 19:44:59,2022-10-17 19:44:59,0,...,Fall,0.0,0.0,19.616667,17.0,2022-10-17,2022-10-17 19:37:00,2022-10-17 20:31:00,20.516667,0
4,30285266,1827,2022-05-12,37,101,1900-01-01 00:00:00,2022-05-12 06:06:00,1900-01-01 00:00:00,2022-05-12 06:13:37,0,...,Spring,0.0,0.0,6.100000,12.0,2022-05-12,2022-05-12 06:06:00,2022-05-12 07:49:00,7.816667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34001818,32467093,4692,2023-05-03,848,114,2023-05-03 22:08:00,1900-01-01 00:00:00,2023-05-03 22:08:57,1900-01-01 00:00:00,57,...,Spring,0.0,0.0,21.383333,3.0,2023-05-03,2023-05-03 21:23:00,2023-05-03 22:08:00,22.133333,0
34001819,32466051,2409,2023-05-03,734,172,2023-05-03 11:41:00,1900-01-01 00:00:00,2023-05-03 11:46:04,1900-01-01 00:00:00,304,...,Spring,0.0,0.0,9.050000,3.0,2023-05-03,2023-05-03 09:03:00,2023-05-03 11:41:00,11.683333,0
34001820,32420846,7868,2023-04-25,848,114,2023-04-25 07:53:00,1900-01-01 00:00:00,2023-04-25 07:56:48,1900-01-01 00:00:00,228,...,Spring,0.0,0.0,6.966667,25.0,2023-04-25,2023-04-25 06:58:00,2023-04-25 07:53:00,7.883333,1
34001821,32420498,6059,2023-04-25,723,121,2023-04-25 11:47:00,1900-01-01 00:00:00,2023-04-25 11:44:48,1900-01-01 00:00:00,-131,...,Spring,0.0,0.0,9.866667,25.0,2023-04-25,2023-04-25 09:52:00,2023-04-25 11:47:00,11.783333,0


In [9]:
stops.dtypes

REF_TOD_TRAIN_ID                          int64
TRAIN_NUMBER                              int64
TRAIN_DEPARTURE_DATE             datetime64[ns]
REF_PTCAR_NO                              int64
PTCAR_ORD_NO                              int32
THR_ARR                          datetime64[ns]
THR_DEP                          datetime64[ns]
HR_ARR                           datetime64[ns]
HR_DEP                           datetime64[ns]
EH_ARR                                    int64
EH_DEP                                    int64
COMMERCIAL_STOP_NATURE                    int16
DEPARTURE_REF_LINE_SECTION_NO             int64
LINE                                    float64
IDENTIFICATOR                            object
YEAR_DEP                                  int32
month                                   float64
season                                   object
weekend                                 float64
holiday                                 float64
dep_hour_origin                         

In [10]:

stops.to_parquet("~/Downloads/all_stops_ungrouped/all_stops_ungrouped.parquet")